# Introduction

Every spring, [Kaggle](https://www.kaggle.com/), a website focused on data science competitions, runs a "March Machine Learning Madness" contest. As is implied by the title, the aim of the contest is to make the best prediction of the results of the NCAA Division I Men's Basketball Tournament. This last year was my first attempt at the competition; unfortunately, due to an error in my program that was left unnoticed, I did not do as well as I had hoped. I was hoping to write this up a long time ago, however, I was unable to due to various school and work commitments. This first part focuses on the advanced metrics and statistics that I used to make my predictions, as well as the initial steps of consolidating and parsing the data.

# First Steps

In [1]:
import numpy as np
import pandas as pd
import random, math
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

After importing the basic foundational modules for the program, we can now define the essential variables. These next two dictionaries will store the ELO ratings and statistics for all the teams.

In [2]:
team_elos = {}
team_stats = {}

Now we can import our data. We will want to use Kaggle's "detailed" results, which will give us access to more advanced statistics, which we will eventually use to create aggregate and averaged statistics.

In [3]:
df = pd.read_csv("data/RegularSeasonDetailedResults.csv")
df.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Lfga3,Lftm,Lfta,Lor,Ldr,Last,Lto,Lstl,Lblk,Lpf
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


We have a few specific columns of this dataframe that are particularly relevant to this analysis. Let's print out the column names to observe.

In [4]:
df.columns

Index(['Season', 'Daynum', 'Wteam', 'Wscore', 'Lteam', 'Lscore', 'Wloc',
       'Numot', 'Wfgm', 'Wfga', 'Wfgm3', 'Wfga3', 'Wftm', 'Wfta', 'Wor', 'Wdr',
       'Wast', 'Wto', 'Wstl', 'Wblk', 'Wpf', 'Lfgm', 'Lfga', 'Lfgm3', 'Lfga3',
       'Lftm', 'Lfta', 'Lor', 'Ldr', 'Last', 'Lto', 'Lstl', 'Lblk', 'Lpf'],
      dtype='object')

As can be seen, the majority of the columns in here correspond to stats, formatted with either "W" or "L" before the stat's name. Let's construct a list to hold the un-modified versions of the stats.

In [5]:
stats = ["fgm", "fga", "fgm3", "fga3", "ftm", "fta", "or", "dr", "ast", "to", "stl", "blk", "pf"]

The eventual goal with these statistics is to calculate averaged values over a certain time-frame before every game. This would provide us with a way of determining the current performance for the team, which will eventually be used in the actual regression. Before we do so, however, we can construct more aggregate statistcs that can be used as a more comprehensive indicator of performance.

# Advanced Statistics

### Effective Field Goal Percentage

The first non-trivial statistic which I will be using is "effective field goal percentage", which I may also refer to as "eFG%". Effective field goal percentage combats one of the major pitfals of standard measures for measuring shooting, accounting for the added difficulty and possible reward of shooting three-pointers. EFG% is calculated like so:

$$\text{eFG}\% = \frac{\text{FGM} + 0.5 \cdot \text{3FGM}}{\text{FGA}}$$

For example, suppose that we have two teams. Team 1 has attempted 10 two-pointers, completing 6 of them. Team 2 has attempted 6 two-pointers, making 4, and 4 two-pointers, making 2. Both teams have attempted ten shots and completed ten of them, resulting in a field goal percentage of 60 percent for both. However, observe what happens when we calculate the eFG% for both:

$$\text{Team 1 eFG}\% = \frac{6 + 0.5 \cdot 0}{10} = 60\%$$

$$\text{Team 2 eFG}\% = \frac{6 + 0.5 \cdot 2}{10} = 70\%$$

As can be seen, even though both teams have the same field goal percentage, Team 2 would have a higher effective field goal percentage due to their additional three-point shooting. This more accurately represents their superior shooting as a whole. 

Our implementation of this in python is simple. Using parameters from the stats list given above, we can define the following function to compute eFG%.

In [6]:
def efg(fgm, fgm3, fga):
    return(fgm + 0.5 * fgm3) / fga

Let's plug in the numbers that we calculated earlier to make sure it works.

In [7]:
efg(6, 0, 10)

0.6

In [8]:
efg(6, 2, 10)

0.7

As can be seen, this gives the proper results.

### Pace

Pace is a measure of the speed of a game, which correlates to the number of possessions by a team in a game. The basic methodology for finding this is rather simple; just sum up the possessions of both the teams. However, in practice, it's a bit more complicated, as it has to account for overtimes. First of all, however, notice that there is no column for possessions in the table. As such, we have to find a way to estimate possessions. To do so, I will use an estimate from [this][pace] website, which is as follows:
[pace]: https://captaincalculator.com/sports/basketball/pace-factor-calculator/

$$\text{Possessions} = \text{FGA} + 0.4 \cdot \text{FTA} - 1.07 \cdot (\frac{\text{ORB}}{\text{ORB} + \text{Opp DRB}}) \cdot (\text{FGA} - \text{FG}) + \text{TOV}$$

Ideally, we want to average out the possessions of both teams to get a more accurate representation. Let's quickly define a function that calculates this so we can use it later.

In [9]:
def possessions(fga, fg, fta, to, orb, opp_drb):
    return (fga + 0.4 * fta - 1.07 * (orb / (orb + opp_drb)) * (fga - fg) + to)

Now that we have this, we can provide a more formal definition of pace, which is

$$\text{Pace} = 40 \cdot \frac{\text{Tm Possessions} + \text{Opp Possessions}}{2 \cdot \text{Game Length}}$$

Observe that the game length is not necessarily 40 minutes at all times; there can be additional five-minute overtimes. Thus, we need to take this into account when writing the function for it, and pass the number of overtimes as a parameter. We thus have:

In [10]:
def pace(poss, opp_poss, ots):
    return 40 * (poss + opp_poss)/(2 * (40 + 5 * ots))

### Offensive and Defensive Ratings

The final sabermetric stats that I will include in my analysis are the offensive and defensive ratings. These stats are both very simple: simply the number of points scored or given up in 100 possessions, like so:

$$\text{Offensive Rating} = 100 \cdot \frac{\text{Points Scored}}{\text{Possessions}}$$

$$\text{Defensive Rating} = 100 \cdot \frac{\text{Points Scored}}{\text{Opp Possessions}}$$

For the sake of this project, I will simply define one rating function that can take either offensive or defensive points and possessions.

In [11]:
def rating(pts, poss):
    return 100 * (pts / poss)

### ELO Ratings

Our final measure of overall team performance will be an ELO rating system, similar to the one used in competitive chess. The basic structure of the ratings are simple: the higher the rating, the better. In addition, the predicted win expectancy depends on the difference between the ELOs of the two teams. After each game, the ELO ratings are updated, with the winner's rating increasing by a certain amount and the loser's decreasing by that same amount, which is higher based on the difference between the initial ratings. In addition, the formula also contains a constant known as the "K-Factor", which controls how responsive the ratings are to wins and losses. From [this][elo] website, I took the K-factor to be equal to 20, which is the optimal number for the NBA. From this, I defined the ELO rating calculation function to be:

[elo]: https://fivethirtyeight.com/features/how-we-calculate-nba-elo-ratings/

In [12]:
def calc_elo(winner, loser):
    rank_diff = winner - loser
    # Calculates the win expectancy.
    exp = (rank_diff * -1) / 400
    odds = 1 / (1 + math.pow(10, exp))
    k = 20
    new_winner_rank = round(winner + (k * (1 - odds)))
    new_rank_diff = new_winner_rank - winner
    new_loser_rank = loser - new_rank_diff
    return new_winner_rank, new_loser_rank

If you would like to play around with sample numbers for the ELO calculator, I have made the following widget below so you can see what happens with various starting ELOs.

In [13]:
from ipywidgets import widgets
input1 = widgets.Text(description="Team 1 Elo: ")
input2 = widgets.Text(description = "Team 2 Elo: ")
button = widgets.Button(description="Submit")
display(input1)
display(input2)
display(button)
def func(sender):
    if input1.value.isdigit() and input2.value.isdigit():
        elo_1 = int(input1.value)
        elo_2 = int(input2.value)
        if abs(elo_1 - elo_2) > 123301:
            return
        values = calc_elo(elo_1, elo_2)
        input1.value = str(values[0])
        input2.value = str(values[1])

button.on_click(func)

# Initializing Data

Now that we have the definitions for all of our statistics, we can now move on to calculating them for every game. To start off with, let's remind ourselves of our current statistics list:

In [14]:
print(stats)

['fgm', 'fga', 'fgm3', 'fga3', 'ftm', 'fta', 'or', 'dr', 'ast', 'to', 'stl', 'blk', 'pf']


We need to add the sabermetric statistics to the list for each row now. Let's define a list of these statistics and generate them.

In [15]:
sabermetrics = ["efg", "ortg", "drtg", "pace"]
for letter in ["W", "L"]:
    for stat in sabermetrics:
        df[letter + stat] = 0
df.columns

Index(['Season', 'Daynum', 'Wteam', 'Wscore', 'Lteam', 'Lscore', 'Wloc',
       'Numot', 'Wfgm', 'Wfga', 'Wfgm3', 'Wfga3', 'Wftm', 'Wfta', 'Wor', 'Wdr',
       'Wast', 'Wto', 'Wstl', 'Wblk', 'Wpf', 'Lfgm', 'Lfga', 'Lfgm3', 'Lfga3',
       'Lftm', 'Lfta', 'Lor', 'Ldr', 'Last', 'Lto', 'Lstl', 'Lblk', 'Lpf',
       'Wefg', 'Wortg', 'Wdrtg', 'Wpace', 'Lefg', 'Lortg', 'Ldrtg', 'Lpace'],
      dtype='object')

As can be seen, we have now initialized all of the columns for the advanced statistics (including possessions for ease of future access). We now need to calculate these statistics for every row. To do so, let's loop through all the rows in the dataframe, calculate the statistics for each, and store them in the corresponding row of the dataframe.

In [16]:
for index, row in df.iterrows():
    ots = row["Numot"]
    letters = ["W", "L"]
    poss_list = []
    for num in range(2):
        letter = letters[num]
        fga = row[letter + "fga"]
        fgm = row[letter + "fgm"]
        fgm3 = row[letter + "fgm3"]
        fta = row[letter + "fta"]
        to = row[letter + "to"]
        orb = row[letter + "or"]
        opp_drb = row[letters[(num + 1) % 2] + "dr"]
        # Calculate stats and place them in list
        poss = possessions(fga, fgm, fta, to, orb, opp_drb)
        poss_list.append(poss)
        df.loc[index, letter + "efg"] = efg(fgm, fgm3, fga)
    avg_poss = sum(poss_list) / 2
    for letter in letters:
        score = row[letter + "score"]
        opp_score = opp_drb = row[letters[(num + 1) % 2] + "score"]
        df.loc[index, letter + "ortg"] = rating(score, avg_poss)
        df.loc[index, letter + "drtg"] = rating(opp_score, avg_poss)
        df.loc[index, letter + "pace"] = pace(avg_poss, avg_poss, ots)

In [17]:
df.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Lblk,Lpf,Wefg,Wortg,Wdrtg,Wpace,Lefg,Lortg,Ldrtg,Lpace
0,2003,10,1104,68,1328,62,N,0,27,58,...,2,20,0.491379,93.570681,93.570681,72.672337,0.433962,85.314444,93.570681,72.672337
1,2003,10,1272,70,1393,63,N,0,26,62,...,6,16,0.483871,102.953162,102.953162,67.992083,0.402985,92.657846,102.953162,67.992083
2,2003,11,1266,73,1437,61,N,0,24,58,...,5,23,0.482759,113.828058,113.828058,64.131815,0.321918,95.116597,113.828058,64.131815
3,2003,11,1296,56,1457,50,N,0,18,38,...,3,23,0.513158,96.720211,96.720211,57.898964,0.428571,86.357332,96.720211,57.898964
4,2003,11,1400,77,1208,71,N,0,30,61,...,1,14,0.540984,122.688411,122.688411,62.760614,0.435484,113.128275,122.688411,62.760614


As can be seen, we now have calculated all of the various sabermetric ratings for every team at every game. Our next official step is going to be to set the ELOs. However, before doing so, we need to initialize the columns in the data for the average statistics. Let's just do so quickly:

In [18]:
all_stats = stats + sabermetrics + ["elo"]
for letter in ["W", "L"]:
    for stat in all_stats:
        if stat != "elo":
            stat = stat + "Avg"
        df[letter + stat] = 0    

Now that we have that all finished, we can set the elos. We want our default elo rating to be 1500. Now, we have two other cases: there exists elo data for the previous game, or it is the first game of the new season and data exists for the season before. If there exists data for the current season, we can just take this for our initial value in our elo calculation. This needs to be tweaked if it is the start of a new season; as a result, I will scale the data in a way that keeps the data centered roughly around 1500, while still relying partially on the performance of the previous season. We thus need to write an accessor method for this. First, recall that at the start of our program we defined a "team_elos" dictionary; we will use this to store all of our values. 

In [19]:
for season in range(2003, 2018):
    team_elos[season] = {}

In [20]:
def get_elo(season, team):
    try:
        return team_elos[season][team]
    except:
        try:
            team_elos[season][team] = round(1 / 2 * team_elos[season - 1][team]) + 1 / 2 * 1496
            return team_elos[season][team]
        except:
            return 1500

We are now able to set all of the elos.

In [21]:
df["Welo"] = 0
df["Lelo"] = 0
for season in range(2003, 2018):
    team_elos[season] = {}
for index, row in df.iterrows():
        w_team = int(row["Wteam"])
        l_team = int(row["Lteam"])
        season = int(row["Season"])
        w_elo = get_elo(season, w_team)
        l_elo = get_elo(season, l_team)
        df.set_value(index, "Welo", w_elo)
        df.set_value(index, "Lelo", l_elo)
        w_elo, l_elo = calc_elo(w_elo, l_elo)
        team_elos[season][w_team] = w_elo
        team_elos[season][l_team] = l_elo

For fun, let's find what team has the best elo in our data.

In [22]:
df.iloc[[df["Welo"].idxmax()]]

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,LastAvg,LtoAvg,LstlAvg,LblkAvg,LpfAvg,LefgAvg,LortgAvg,LdrtgAvg,LpaceAvg,Lelo
65868,2015,132,1246,78,1116,63,N,0,25,49,...,0,0,0,0,0,0,0,0,0,1672


This is the 2015 Kentucky Wildcats, one of the best regular-season teams of recent memory. As of right now, our elo rating seems to be adequate. Finally, to complete our aggregate stores, we just need to find a way to average and save the statistics for every game. We need to choose some sort of benchmark number of games to use to caclculate the running average. I will start off with 10 as that number, but can change it later on when tweaking the model.

In [23]:
AVG_COUNT = 10
# We don't want elo scores in our list of averaged stats, so let's remove it from the list.
all_stats.remove("elo")
for season in range(2003, 2018):
    for team in range(1100, 1465):
        team_data = df[(df["Wteam"] == team) | (df["Lteam"] == team)]
        team_data = team_data[team_data["Season"] == season]
        for stat in all_stats:
            stat_list = []
            for index, row in team_data.iterrows():
                if team_data.get_value(index, "Wteam") == team:
                    win_char = "W"
                else:
                    win_char = "L"
                stat_list.append(team_data.get_value(index, win_char + stat))
                if len(stat_list) > AVG_COUNT:
                    stat_list.pop(0)
                    stat_avg = sum(stat_list)/len(stat_list)
                    df.loc[index, win_char + stat + "Avg"] = stat_avg     

In [24]:
df.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,LastAvg,LtoAvg,LstlAvg,LblkAvg,LpfAvg,LefgAvg,LortgAvg,LdrtgAvg,LpaceAvg,Lelo
0,2003,10,1104,68,1328,62,N,0,27,58,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1500
1,2003,10,1272,70,1393,63,N,0,26,62,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1500
2,2003,11,1266,73,1437,61,N,0,24,58,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1500
3,2003,11,1296,56,1457,50,N,0,18,38,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1500
4,2003,11,1400,77,1208,71,N,0,30,61,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1500


Notice that the first ten games for each team for each season have zeros for the statistics. We thus need to drop these games from our final averaged results. To do so, we just need to select all rows such that some stat's averages are valid numbers.

In [25]:
parsed_data = df[(df["WfgaAvg"] != 0) & (df["LfgaAvg"] != 0)]

In [26]:
parsed_data.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,LastAvg,LtoAvg,LstlAvg,LblkAvg,LpfAvg,LefgAvg,LortgAvg,LdrtgAvg,LpaceAvg,Lelo
1297,2003,54,1182,71,1198,68,A,1,27,56,...,10.2,18.8,9.3,2.7,19.8,0.448533,89.090745,102.665032,67.659421,1478
1335,2003,54,1337,116,1114,110,N,2,41,84,...,13.5,17.7,6.9,3.0,18.7,0.541350,103.058254,109.588386,69.476500,1517
1355,2003,54,1390,81,1211,71,N,0,29,57,...,16.6,15.7,5.1,3.5,18.6,0.543031,111.970235,116.432606,69.981853,1545
1383,2003,55,1160,76,1260,72,N,0,27,55,...,12.9,13.9,7.4,3.2,22.5,0.470060,102.042134,115.081433,72.342199,1472
1407,2003,56,1108,74,1410,56,N,0,28,50,...,10.3,14.8,8.4,1.4,17.7,0.389098,85.551340,107.931222,63.999786,1429


As can be seen, all of our statistics are now valid, so we can now write it to a file and save it.

In [27]:
df.to_csv("data/averaged_data.csv" index=False)

This concludes the initial part of our data analysis; from this point on, we will not be creating any more features, and will instead move on into the direct analysis and prediction part of our project. To summarize, up to this point, we have:

1. Defined advanced statistics and wrote implementations for them
2. Used those statistics to generate more features from the initial data given to us.
3. Parsed the data and presented it in a way that will be accessible to our prediction model.

In the next part, I will pick up from where I left off, and go through my initial steps in forming a simple prediction model. 